In [1]:
import json
import matplotlib.pyplot as plt

from src.data_processing.data_loader import data_loader
import src.training.training_utils as training_utils
# from src.results.main import evaluate
# import src.visualisation.main as vis_main


In [2]:
with open("src/training/run_config.json", "r") as f:
    run_config = json.load(f)
    f.close()

# Extract info
data_config = run_config["data_config"]
model_config = run_config["model_config"]
training_config = run_config["training_config"]

In [3]:
data_info = data_loader(**data_config);

C:\Users\hruia\PycharmProjects\VarPhenClustering\src\data_processing\data_loading_utils.py:453: FutureWarning: is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.
  cond1 = X[self.id_col].is_monotonic



MIMIC data has been subsettted to the following features: 
 ['DBP', 'ESI', 'HR', 'RR', 'SBP', 'SPO2', 'TEMP', 'age', 'gender'].


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6642/6642 [00:24<00:00, 275.14it/s]



MIMIC data successfully loaded.

Basic information 
 Input shape: (6642, 10, 9), (6642, 4) 
 Outcome Distribution: De      21
I     1097
W     5385
Di     139
dtype: int64


In [32]:
%load_ext autoreload
%autoreload 2

from importlib import reload
from src.models.Dir_VRNN import Dir_VRNN_train
from src.models.Dir_VRNN import model
from src.models.Dir_VRNN import Dir_VRNN_utils

reload(Dir_VRNN_train);
reload(model);
reload(Dir_VRNN_utils);


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
model_config["num_clus"] = 2
model = Dir_VRNN_train.DirVRNN(data_info = data_info, model_config=model_config, training_config=training_config);

In [34]:

# Train model
model.fit()


Train epoch: 1   [-120.57631 - 100%]
Val epoch: 1   [-130.28052 - 0%]
Train epoch: 2   [-148.24089 - 100%]
Val epoch: 2   [-175.20398 - 0%]
Train epoch: 3   [-204.31955 - 100%]
Val epoch: 3   [-243.35699 - 0%]
Train epoch: 4   [-315.60633 - 100%]
Val epoch: 4   [-448.58313 - 0%]
Train epoch: 5   [-728.04981 - 100%]
Val epoch: 5   [-1217.80762 - 0%]


In [35]:
 
    
# Predict model
X_test, y_test = data_info["X"][-1], data_info["y"][-1]
history = model.predict(X_test, y_test)
     
# Print available keys
print(history.keys())
             
                 
                     
                         
                             
                                 
                                     
                                         
                                             
                                                 
                                                    

Test: [-1219.08655 - 0%]
dict_keys(['alpha_prior', 'alpha_enc', 'est_pi', 'est_cluster_means', 'est_outcomes', 'est_gen_mean', 'est_gen_data', 'cell_state', 'pred_loss', 'y_pred', 'X', 'y'])


In [42]:
import numpy as np
history["alpha_enc"][0, :, :] / torch.sum(history["alpha_enc"][0, :, :], dim=-1, keepdims=True)

tensor([[0.3198, 0.6802],
        [0.3029, 0.6971],
        [0.3031, 0.6969],
        [0.3030, 0.6970],
        [0.3031, 0.6969],
        [0.3031, 0.6969],
        [0.3031, 0.6969],
        [0.3030, 0.6970],
        [0.3030, 0.6970],
        [0.3030, 0.6970]], grad_fn=<DivBackward0>)

In [43]:
history["alpha_enc"][101, :, :] / torch.sum(history["alpha_enc"][101, :, :], dim=-1, keepdims=True)

tensor([[0.3194, 0.6806],
        [0.3027, 0.6973],
        [0.3031, 0.6969],
        [0.3030, 0.6970],
        [0.3031, 0.6969],
        [0.3032, 0.6968],
        [0.3020, 0.6980],
        [0.3020, 0.6980],
        [0.3020, 0.6980],
        [0.3020, 0.6980]], grad_fn=<DivBackward0>)

In [38]:
import torch
torch.min(history["est_pi"], dim=1)

torch.return_types.min(
values=tensor([[0.3029, 0.6802],
        [0.3025, 0.6785],
        [0.3039, 0.6778],
        ...,
        [0.3020, 0.6792],
        [0.3020, 0.6800],
        [0.3014, 0.6812]], grad_fn=<MinBackward0>),
indices=tensor([[1, 0],
        [3, 0],
        [1, 0],
        ...,
        [8, 0],
        [8, 0],
        [8, 0]]))